In [ ]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from numpy import load
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from itertools import chain

from projectwind.data import get_data
from projectwind.clean import clean_timesteps

from sklearn import set_config; set_config(display='diagram')

# LSTM_data.py

In [ ]:
data = get_data(25)

In [ ]:
save = data.copy()

## Quick check on number of NaN over the period

In [ ]:
# Count number of na
isna_df = data[0].isna().sum(axis=1) / len(data[0].columns)

# Resample on daily basis
daily_data = isna_df.resample('D').sum() / 6 # divide by # periods to get ~ 24hr % missing values

# Graph output
fig, ax = plt.subplots(figsize=(20,5))
plt.bar(x=daily_data.index, height=daily_data.values);

In [ ]:
# Good to see there are less missing values towards end of data 
# (meaning test and validation sets should be of better quality than train set)

## Data engineering

### Interpolate all NaNs

In [ ]:
from projectwind.LSTM_preproc import clean_LSTM_data

In [ ]:
full_data = clean_LSTM_data(data)

### Create wind speed & direction vectors and misalignment to average wind farm vector

In [ ]:
# Average wind speeds & directions over the wind turbines
wind_speeds = pd.DataFrame()
nacelle_dir = pd.DataFrame()
misalignment = pd.DataFrame()
for idx, WTG_data in enumerate(data):
    wind_speeds[idx] = WTG_data['Wind Speed']
    nacelle_dir[idx] = WTG_data['Nacelle Orientation']
    misalignment[idx] = WTG_data['Misalignment']

In [ ]:
for idx in misalignment:
    misalignment[idx] = misalignment[idx].apply(lambda x: x if x <=180 else (360 - x)*-1)

In [ ]:
wind_dir = pd.DataFrame()
for idx in misalignment:
    wind_dir[idx] = nacelle_dir[idx] - misalignment[idx]

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(15, 15))
for i in range(len(nacelle_dir.columns)):
    row, col = i//5, i%5
    axes[row,col].hist2d(wind_dir[i], wind_speeds[i], bins=(50, 50), vmax=400)
    plt.xlabel('Wind Direction [deg]')
    plt.ylabel('Wind Velocity [m/s]');
plt.tight_layout()

In [ ]:
wind_X_vector = pd.DataFrame()
wind_Y_vector = pd.DataFrame()
for idx in wind_dir:
    wind_dir[idx] = wind_dir[idx] * np.pi / 180 # convert to radians
    wind_X_vector[idx] = wind_speeds[idx] * np.cos(wind_dir[idx])  # get X vector
    wind_Y_vector[idx] = wind_speeds[idx] * np.sin(wind_dir[idx])  # get Y vector
    wind_X_vector[idx] = wind_speeds[idx] * np.cos(wind_dir[idx])  # get X vector
    wind_Y_vector[idx] = wind_speeds[idx] * np.sin(wind_dir[idx])  # get Y vector

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(15, 15))
for i in range(len(nacelle_dir.columns)):
    row, col = i//5, i%5
    axes[row,col].hist2d(wind_X_vector[i], wind_Y_vector[i], bins=(50, 50), vmax=400)
    plt.xlabel('Wind X [m/s]')
    plt.ylabel('Wind Y [m/s]');
plt.tight_layout()

In [ ]:
for idx, WTG_data in enumerate(data):
    WTG_data['WTG_wind_X'] = wind_X_vector[idx]
    WTG_data['WTG_wind_Y'] = wind_Y_vector[idx]
#     WTG_data['Misalign_wind_X'] = average['X_vector'] - wind_X_vector[idx] # Creates data leakage!
#     WTG_data['Misalign_wind_Y'] = average['Y_vector'] - wind_Y_vector[idx] # Creates data leakage!
    WTG_data.drop(columns=['Misalignment','Nacelle Orientation'], inplace=True)

### Power vs. Rotor Speed check

In [ ]:
temp = data[19].loc['2020-03-05':'2020-03-06']

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sns.lineplot(x=temp.index, y=temp['Power']/150)
sns.lineplot(x=temp.index, y=temp['Rotor Speed'])
sns.lineplot(x=temp.index, y=temp['Wind Speed'])
fig.legend(['Power','Rotor Speed', 'Wind Speed']);

### Sine/Cosine Time

In [ ]:
df = pd.DataFrame(index=data[0].index)
timestamp_s = data[0].index.map(pd.Timestamp.timestamp)

day = 24*60*60

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))

In [ ]:
plt.plot(df.iloc[0:792,0])
plt.plot(df.iloc[0:792,1]);

In [ ]:
for WTG_data in data:
    WTG_data['Day sin'] = df['Day sin']
    WTG_data['Day cos'] = df['Day cos']

## Split the data

In [ ]:
df = data[0]
df.head()

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
df_std = (df - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

## Data Windowing

In [ ]:
import tensorflow as tf

In [ ]:
df.head()

### Example

In [ ]:
data[2]

In [ ]:
df.columns

In [43]:
w1 = WindowGenerator(input_width=6, label_width=1, shift=1,
                     label_columns=df.columns)
w1

NameError: name 'WindowGenerator' is not defined

In [ ]:
w2 = WindowGenerator(input_width=24*5*6, label_width=12*6, shift=12*6,
                     label_columns=['Power'])
w2

In [ ]:
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
# Each element is an (inputs, label) pair.
w2.train.element_spec

In [ ]:
for i in w2.train.take(25):
    print(i)

In [ ]:
for example_inputs, example_labels in w2.train.take(100):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
dataset = w2.train.take(1)

In [ ]:
for sequence in dataset:
    sequence.plot()

### Adaptation

In [ ]:
from projectwind.LSTM_data import WindowGenerator, get_LSTM_data, define_window

In [ ]:
train_df, val_df, test_df = get_LSTM_data(2)

In [ ]:
def make_dataset(data):
    X_datasets = []
    y_datasets = []
    
    for WTG_data in data:
        
        # Find sequences according to window size of X and y
        WTG_data = np.array(WTG_data, dtype=np.float32)
        WTG_sequences = tf.keras.utils.timeseries_dataset_from_array(data=WTG_data,
                                                                    targets=None,
                                                                    sequence_length=window.total_window_size,
                                                                    sampling_rate=1,
                                                                    sequence_stride=window.total_window_size,
                                                                    shuffle=False,
                                                                    batch_size=32)
        # Split X and y according to window size
        WTG_sequences = WTG_sequences.map(window.split_window)
        
        # Transfer from tensor to numpy array to save under .NPY format
        X_datasets.append(chain.from_iterable([X.numpy() for X, y in WTG_sequences]))
        y_datasets.append(chain.from_iterable([y.numpy() for X, y in WTG_sequences]))
        
    # Aggregate WTGs batches into same array
    X_array = np.array(list(chain.from_iterable(X_datasets)))
    y_array = np.array(list(chain.from_iterable(y_datasets)))
    
    #X_array, y_array = shuffle_sequences(X_array, y_array)
        
    return X_array, y_array

In [ ]:
# Make datasets
X_train, y_train = make_dataset(train_df)
X_val, y_val = make_dataset(val_df)
X_test, y_test = make_dataset(test_df)

In [ ]:
# Verify dataset shapes
print(f"Train set shape:  X: {X_train.shape}, Y: {y_train.shape}")
print(f"Val set shape:    X: {X_val.shape},  Y: {y_val.shape}")
print(f"Test set shape:   X: {X_test.shape},  Y: {y_test.shape}")

In [ ]:
def shuffle_sequences(X, y, seed=42):
    np.random.seed(seed)
    np.random.shuffle(X)
    np.random.seed(seed)
    np.random.shuffle(y)
    return X, y

In [ ]:
X_train, y_train = shuffle_sequences(X_train, y_train, seed=1)
X_val, y_val = shuffle_sequences(X_val, y_val, seed=2)
X_test, y_test = shuffle_sequences(X_test, y_test, seed=3)

In [ ]:
sequence_name = f"{window.input_width // 6}-{window.label_width//6}"
np.save(f'./projectwind/data/LSTM_sequence_X_train_{sequence_name}.npy', np.asanyarray(X_train, dtype=object))
np.save(f'./projectwind/data/LSTM_sequence_y_train_{sequence_name}.npy', np.asanyarray(y_train, dtype=object))
np.save(f'./projectwind/data/LSTM_sequence_X_val_{sequence_name}.npy', np.asanyarray(X_val, dtype=object))
np.save(f'./projectwind/data/LSTM_sequence_y_val_{sequence_name}.npy', np.asanyarray(y_val, dtype=object))
np.save(f'./projectwind/data/LSTM_sequence_X_test_{sequence_name}.npy', np.asanyarray(X_test, dtype=object))
np.save(f'./projectwind/data/LSTM_sequence_y_test_{sequence_name}.npy', np.asanyarray(y_test, dtype=object))

In [ ]:
train_ds = np.load(f'./projectwind/data/LSTM_sequence_train_datasets.npy', allow_pickle=True)

In [ ]:
len(train_ds)

In [ ]:
for inputs, labels in train_ds:
    print(f'Inputs shape (batch, time, features): {inputs.shape}')
    print(f'Labels shape (batch, time, features): {labels.shape}')

### Test of python file

In [ ]:
from projectwind.LSTM_data import WindowGenerator, get_LSTM_data, define_window

In [ ]:
train_df, val_df, test_df = get_LSTM_data(2)

In [ ]:
len(train_df[0])

In [82]:
n_steps_in = 5 * 24 * 6     # 5 day x 24hrs x 6 periods of 10min
n_steps_out = 12 * 6    # 12hours x 6 periods of 10min
window = define_window(n_steps_in, n_steps_out, train_df[0], val_df[0], test_df[0])

In [84]:
for inputs, labels in window.train.take(1):
    print(f'Inputs shape (batch, time, features): {inputs.shape}')
    print(f'Labels shape (batch, time, features): {labels.shape}')

Inputs shape (batch, time, features): (32, 720, 9)
Labels shape (batch, time, features): (32, 72, 1)


# LSTM_model.py

## Load Dataset

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from projectwind.LSTM_data import get_LSTM_data, define_window, load_datasets

In [4]:
train_df, val_df, test_df = get_LSTM_data(2)

/home/shmiggit/code/AmaurySalles/projectwind/raw_data
1
2


In [5]:
n_steps_in = 5 * 24 * 6     # 5 day x 24hrs x 6 periods of 10min
n_steps_out = 12 * 6    # 12hours x 6 periods of 10min
window = define_window(n_steps_in, n_steps_out, train_df, val_df, test_df)

In [7]:
X_train, y_train = window.train
X_val, y_val = window.val
X_test, y_test = window.test

In [39]:
train_ds = window.train
val_ds = window.val
test_ds = window.test

In [41]:
train_ds

<MapDataset element_spec=(TensorSpec(shape=(None, 720, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 72, 1), dtype=tf.float32, name=None))>

In [14]:
# Verify dataset shapes
print(f"Train set shape:  X: {X_train.shape},  Y: {y_train.shape}")
print(f"Val set shape:    X: {X_val.shape},  Y: {y_val.shape}")
print(f"Test set shape:   X: {X_test.shape},  Y: {y_test.shape}")

Train set shape:  X: (224, 720, 9),  Y: (224, 72, 1)
Val set shape:    X: (64, 720, 9),  Y: (64, 72, 1)
Test set shape:   X: (32, 720, 9),  Y: (32, 72, 1)


## Create feedback_model 

In [85]:
class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_features)

    def warmup(self, inputs):
        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(inputs)

        # predictions.shape => (batch, features)
        prediction = self.dense(x)
        return prediction, state

In [87]:
for inputs, labels in window.train.take(1):
    num_features = inputs.shape[2]
    n_steps_in = inputs.shape[1]
    n_steps_out = labels.shape[1]

In [88]:
feedback_model = FeedBack(units=32, out_steps=n_steps_out)

In [89]:
prediction, state = feedback_model.warmup(window.example[0])
prediction.shape

TensorShape([32, 9])

In [90]:
feedback_model.summary

<bound method Model.summary of <__main__.FeedBack object at 0x7f0abcac60d0>>

# OLD - LSTM_data_preproc

In [ ]:
raw_data = get_data(25)

In [ ]:
raw_data[13]

In [ ]:
data = clean_timesteps(raw_data)
data = clean_LSTM_data(data)

## clean_LSTM_data

In [ ]:
temp_data = data[0]

In [ ]:
index_with_nan = temp_data[temp_data.isna().any(axis=1) == True].index

In [ ]:
temp_data.loc[index_with_nan]

In [ ]:
temp_data.loc['2019-05-06 09:30':'2019-05-06 15:00']

In [ ]:
temp_data.loc['2019-05-06 09:30':'2019-05-06 15:00']

In [ ]:
for WTG_data in data:
    print(WTG_data.isna().sum().sum())
    WTG_data.interpolate(axis=0, inplace=True)
    print(WTG_data.isna().sum().sum())

##  split_LSTM_data

In [ ]:
temp = data[0]

In [ ]:
len(data[0])

In [ ]:
seq_len = int(24 * 6 * 5.5)
seq_len

In [ ]:
# Find number of seq possible
seq_num = len(data[0]) // (720+72) # per turbine
seq_num

In [ ]:
test_seq_len = int(0.2 * seq_num) # last 20% indices will belong to test set
val_seq_len  = int(0.2 * seq_num) # 2nd last 20% indices will belong to val set
test_seq_len, val_seq_len

In [ ]:
test_seq_start = seq_num - test_seq_len
val_seq_start = seq_num - test_seq_len - val_seq_len
0, val_seq_start, test_seq_start, 160

In [ ]:
test_idx_start = test_seq_start * seq_len
val_idx_start = val_seq_start * seq_len
val_idx_start, test_idx_start

In [ ]:
# Aggregated
test_idx_start = int((seq_num - (seq_num * 0.2)) * seq_len)
val_idx_start = int((seq_num - (seq_num * 0.4)) * seq_len)
val_idx_start, test_idx_start

In [ ]:
# Aggregated
seq_len = int(24 * 6 * 5.5)
seq_num = len(data[0]) // (720+72) # per turbine
test_idx_start = int(seq_num * (0.8 * seq_len))
val_idx_start = int(seq_num * (0.6 * seq_len))
val_idx_start, test_idx_start

In [ ]:
temp.iloc[val_idx_start:test_idx_start]

In [ ]:
# Test function
from projectwind.LSTM_preproc import split_train_val_test_split

In [ ]:
train, val, test = split_train_val_test_split(data, 5.5)

In [ ]:
train[0]

## get_sequence

In [ ]:
from projectwind.LSTM_preproc import get_sequences

In [ ]:
sequences = get_sequences(train, 5.5)

In [ ]:
sequences

## extract_target_from_sequences

In [ ]:
datasets = {'train':[1,2,3], 'val':[4,5], 'test':[6,7]}

In [ ]:
datasets = dict(train=[1,2,3], val=[4,5], test=[6,7])

In [ ]:
datasets

In [ ]:
for name, data in datasets.items():
    print(name)

In [ ]:
from projectwind.LSTM_preproc import extract_target_from_sequences

In [ ]:
sequences[0].shape[0]

In [ ]:
X, Y = extract_target_from_sequences(sequences, 0.5)

## init_LSTM_data

In [ ]:
from projectwind.LSTM_preproc import init_LSTM_data

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = init_LSTM_data(1, 5.5)

In [ ]:
len(X_train), len(X_val), len(X_test)

# OLD - Trainer_LSTM_model

In [ ]:
train_ds = np.load(f'./projectwind/data/LSTM_sequence_train_datasets.npy', allow_pickle=True)
val_ds = np.load(f'./projectwind/data/LSTM_sequence_val_datasets.npy', allow_pickle=True)
test_ds = np.load(f'./projectwind/data/LSTM_sequence_test_datasets.npy', allow_pickle=True)

In [28]:
from projectwind.LSTM_model import init_LSTM_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization

In [29]:
def init_LSTM_model(n_steps_in, n_steps_out, n_features):

    model = Sequential()
    model.add(BatchNormalization(input_shape=(n_steps_in, n_features)))
    model.add(LSTM(16, activation='tanh', return_sequences=True))
    model.add(LSTM(32, activation='tanh', return_sequences=False))
    model.add(Dense(n_steps_out, activation='linear'))
    model.compile(optimizer='adam', loss='huber', metrics=["mae"])

    return model

In [30]:
model = init_LSTM_model(n_steps_in=window.input_width, 
                        n_steps_out=window.label_width, 
                        n_features=len(window.column_indices))

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 720, 9)           36        
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 720, 16)           1664      
                                                                 
 lstm_1 (LSTM)               (None, 32)                6272      
                                                                 
 dense (Dense)               (None, 72)                2376      
                                                                 
Total params: 10,348
Trainable params: 10,330
Non-trainable params: 18
_________________________________________________________________


In [42]:
# MAE
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=1, restore_best_weights=True)

history = model.fit(window.train,
                    validation_data=window.val,
                    epochs=2,
                    callbacks=[es])

Epoch 1/2
4/4 [==============================] - 3s 505ms/step - loss: 699.1368 - mae: 699.6363 - val_loss: 1054.7458 - val_mae: 1055.2458
Epoch 2/2
4/4 [==============================] - 3s 765ms/step - loss: 699.0745 - mae: 699.5739 - val_loss: 1054.6660 - val_mae: 1055.1660


In [ ]:
# MSE
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=1, restore_best_weights=True)

history = model.fit(X_train,y_train,
                    validation_data=(X_val, y_val),
                    epochs=2,
                    callbacks=[es])

In [ ]:
from projectwind.trainer import plot_loss

In [ ]:
plot_loss(history)

## 